In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import linear_kernel
data = pd.read_csv('book_data.csv')
print(data.head())

                                        book_authors  \
0                                    Suzanne Collins   
1                         J.K. Rowling|Mary GrandPré   
2                                         Harper Lee   
3  Jane Austen|Anna Quindlen|Mrs. Oliphant|George...   
4                                    Stephenie Meyer   

                                           book_desc  \
0  Winning will make you famous. Losing means cer...   
1  There is a door at the end of a silent corrido...   
2  The unforgettable novel of a childhood in a sl...   
3  «È cosa ormai risaputa che a uno scapolo in po...   
4  About three things I was absolutely positive.F...   

                         book_edition book_format    book_isbn book_pages  \
0                                 NaN   Hardcover  9.78044E+12  374 pages   
1                          US Edition   Paperback  9.78044E+12  870 pages   
2                    50th Anniversary   Paperback  9.78006E+12  324 pages   
3  Modern Library 

In [2]:
#I will select three columns from the dataset for the rest of the task (book_title, book_desc, book_rating_count):
data = data[['book_title', 'book_desc', 'book_rating_count']]
print(data.head())

                                  book_title  \
0                           The Hunger Games   
1  Harry Potter and the Order of the Phoenix   
2                      To Kill a Mockingbird   
3                        Pride and Prejudice   
4                                   Twilight   

                                           book_desc  book_rating_count  
0  Winning will make you famous. Losing means cer...            5519135  
1  There is a door at the end of a silent corrido...            2041594  
2  The unforgettable novel of a childhood in a sl...            3745197  
3  «È cosa ormai risaputa che a uno scapolo in po...            2453620  
4  About three things I was absolutely positive.F...            4281268  


In [4]:
#Let’s have a look at the top 5 books in the dataset according to the number of ratings:
data = data.sort_values(by='book_rating_count', ascending=False)
top_5 = data.head()

from matplotlib.pyplot import title
import plotly.express as px
import plotly.graph_objects as go
labels = top_5['book_title']
values = top_5['book_rating_count']
colors = ['gold', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(title_text='Top 5 Rated Books')
fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size= 30,
                marker=dict(colors=colors, line=dict(color='black', width=3)))

fig.show()


In [5]:
#Before moving forward, let’s check if the data has null values or not:
print(data.isnull().sum())

book_title              0
book_desc            1331
book_rating_count       0
dtype: int64


In [6]:
#The dataset has null values in the book description column. Let’s drop the rows having null values:
data = data.dropna()

In [10]:
#Now I will use the book description column as the feature to recommend similar books to the user:
feature = data["book_desc"].tolist()
tfidf = text.TfidfVectorizer(input=feature, stop_words="english")
tfidf_matrix = tfidf.fit_transform(feature)
similarity = linear_kernel(tfidf_matrix, tfidf_matrix)


MemoryError: Unable to allocate 11.8 GiB for an array with shape (1584423582,) and data type float64

In [8]:
#Now I will set the book title column as an index so that we can find similar books by giving the title of the book as an input:
indices = pd.Series(data.index, index=data['book_title']).drop_duplicates()

In [9]:
#Now here’s how to write a function to recommend similar books:
def book_recommendation(title, similarity = similarity):
    index = indices[title]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[0:5]
    bookindices = [i[0] for i in similarity_scores]
    return data['book_title'].iloc[bookindices]

print(book_recommendation("Letters to a Secret Lover"))

NameError: name 'similarity' is not defined

In [ ]:
#Didn't work because there wasn't enough space, i will try to find a solution